In [1]:
import pandas as pd
from src.data.hex_utils import * 
import pickle
import numpy as np

In [2]:
with open("../data/processed/geolife_hex_50.pkl", "rb") as f:
    hdf = pickle.load(f)

In [18]:
hdf.groupby('user')['t_idx'].nunique().describe()

count     175.000000
mean       84.297143
std       205.682946
min         1.000000
25%         6.000000
50%        22.000000
75%        73.500000
max      1717.000000
Name: t_idx, dtype: float64

In [21]:
hdf

,q,r,datetime,user,t_idx,cell_dist,is_workday,is_in_time_0,is_in_time_1,is_in_time_2,is_in_time_3
312042,61,80,2008-10-23 02:53:04.000000000,0,1,1.0,True,True,False,False,False
312042,21,53,2008-10-23 02:53:04.000000000,0,1,1.0,True,True,False,False,False
312054,20,54,2008-10-23 02:54:05.000000000,0,1,1.0,True,True,False,False,False
312109,19,54,2008-10-23 02:58:40.000000000,0,1,1.0,True,True,False,False,False
0,18,55,2008-10-23 03:36:41.000000000,0,1,1.0,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...
17,31,35,2007-12-08 05:55:15.076923076,181,17118,1.0,False,True,False,False,False
12420255,32,34,2007-12-08 05:58:54.000000000,181,17118,1.0,False,True,False,False,False
12420265,32,35,2007-12-08 06:10:43.000000000,181,17118,1.0,False,True,False,False,False
12420277,32,34,2007-12-08 07:03:03.000000000,181,17118,1.0,False,False,True,False,False


In [3]:
test_split = 0.1
val_split = 0.2

stages = ["train", "val", "test"]
results = {stage: [] for stage in stages}
for _, df_user in hdf.groupby("user"):
    trajectory_ids = df_user["t_idx"].unique()
    n_trajectories = len(trajectory_ids)
    n_train = int(n_trajectories * (1 - val_split - test_split))
    n_val = int(n_trajectories * val_split)
    results["train"].append(
        df_user[df_user["t_idx"].isin(trajectory_ids[:n_train])]
    )
    results["val"].append(
        df_user[
            df_user["t_idx"].isin(trajectory_ids[n_train : n_train + n_val])
        ]
    )
    results["test"].append(
        df_user[df_user["t_idx"].isin(trajectory_ids[n_train + n_val :])]
    )

data = {stage: pd.concat(results[stage]) for stage in stages}

In [14]:
data['val'].groupby("user")['t_idx'].nunique().max() / data['val']['t_idx'].nunique()

0.11909722222222222